Short notebook to load and visualize a cloud point after generation (voxelized with voxels of size 10cm) 

In [ ]:
from utils_notebook import load_and_decimate_room_pc_voxels, point_cloud_visu

In [ ]:
CLOUD_POINT_PATH = "./output/pointcloud/point_cloud.ply"

In [ ]:
pts_vox, cols_vox = load_and_decimate_room_pc_voxels(CLOUD_POINT_PATH, 0.1)
point_cloud_visu(pts_vox, cols_vox)